# 0 Move the stills to directories created perviously - This will only move 

In [134]:
import piexif
import glob
import pandas as pd
import shutil
import os
import re
import hashlib
from uvsp.logfile import LogFile
import exiftool

def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()# Step 4 load up all the exif data into a data frame

def process_exifdata(file,dohash=False):
    item =piexif.load(file)
    if piexif.ExifIFD.DateTimeDigitized in item["Exif"]:
        dateTime =item["Exif"][piexif.ExifIFD.DateTimeDigitized].decode("utf-8") 
    if piexif.ExifIFD.SubSecTimeDigitized in item["Exif"]:
        dateTime = dateTime +'.' +item["Exif"][piexif.ExifIFD.SubSecTimeDigitized].decode("utf-8")
    else:
        dateTime = dateTime +'.00'
    dateTime = pd.to_datetime(dateTime,format="%Y:%m:%d %H:%M:%S.%f")
    if piexif.ExifIFD.CameraOwnerName in item["Exif"]:
        name = item["Exif"][piexif.ExifIFD.CameraOwnerName].decode("utf-8")
    else:
        name = "Unknown"   
    if piexif.ExifIFD.BodySerialNumber in item["Exif"]:
        body =int(item["Exif"][piexif.ExifIFD.BodySerialNumber].decode("utf-8"))
    else:
        body =""
    lens = item["Exif"][piexif.ExifIFD.LensModel].decode("utf-8")
    if dohash:
      hashmd5 = md5(file)
    else:
      hashmd5 = None
    return {'timestamp':dateTime,'cameraOwnerName':name,'bodySerialNumber':body,'lens':lens,'origonal_file':file,'process':False,'origonal_file_hash':hashmd5}

def process_exifvideo(file,dohash=False):
    item =piexif.load(file)
    return item

def opendir_dialog(title,default_dir=r'Z:\voyage_specific\MRITC'):
    from PyQt5 import QtGui
    from PyQt5 import QtGui, QtWidgets
    app = QtWidgets.QApplication([dir])
    fname = QtWidgets.QFileDialog.getExistingDirectory(None, title,default_dir)
    return str(fname)

def openfile_dialog(title,default_dir=r'Z:\voyage_specific\MRITC'):
    from PyQt5 import QtGui
    from PyQt5 import QtGui, QtWidgets
    app = QtWidgets.QApplication([dir])
    filename, _filter  = QtWidgets.QFileDialog.getOpenFileName(None, title,default_dir, "MOV (*.MOV)")
    return str(filename)



# Fudge factor for times 

In [135]:
delta_time = pd.to_timedelta(5,'m')

# 1 select the video directory of the images you want to process...

In [136]:
filename = opendir_dialog("select Video file....")

# 2 get the metadata set up

In [137]:
files = glob.glob(filename+os.path.sep+'*.MOV')
with exiftool.ExifTool(r'../exiftool(-k -api largefilesupport=1).exe') as et:
    metadata = et.get_metadata_batch(files)
metadata = pd.DataFrame(metadata)
metadata.head()
metadata['origonal_file'] = filename
metadata['File:FileCreateDate'] = pd.to_datetime(metadata['File:FileCreateDate'].str.split('+')[0][0],format='%Y:%m:%d %H:%M:%S')
metadata['QuickTime:TrackCreateDate'] = pd.to_datetime(metadata['QuickTime:TrackCreateDate'],format='%Y:%m:%d %H:%M:%S')
metadata[['QuickTime:TrackCreateDate','File:FileCreateDate']]

mask =metadata['QuickTime:TrackCreateDate'].isna()
metadata.loc[mask,'QuickTime:TrackCreateDate']= metadata.loc[mask,'File:FileCreateDate']
metadata['QuickTime:Duration'] =pd.to_timedelta(metadata['QuickTime:Duration'],'S')
metadata['endtime'] = metadata['QuickTime:TrackCreateDate'] + metadata['QuickTime:Duration']
print('start',metadata['QuickTime:TrackCreateDate'])
print('end',metadata['endtime'])
starttime = metadata['QuickTime:TrackCreateDate'].min() - delta_time
endtime = metadata['endtime'].max() + delta_time
parts =os.path.split(files[0])[1].split('_')
platform = parts[0]
voyage = parts[2]+'_'+ parts[3]
operation = parts[4]
print('platform  :',platform)
print('Voyage    :',voyage)
print('operation :',operation)
metasource = r'Z:\science\MRITC\METADATA\MRITC_META_IN2018_V06_000_20181123T034451Z.JSON'
log = LogFile(metasource)
log.load()
log.deployment_metadata['Operation_identifier'] = int(operation)

start 0   2018-12-14 12:03:55
1   2018-12-14 12:22:28
Name: QuickTime:TrackCreateDate, dtype: datetime64[ns]
end 0   2018-12-14 12:22:08.440
1   2018-12-14 12:32:35.360
Name: endtime, dtype: datetime64[ns]
platform  : MRITC
Voyage    : IN2018_V06
operation : 165


# 3 Select the Image file directory

In [138]:
sourcedir =opendir_dialog("select Image file Dir for %s...."%(operation),r'Z:\voyage_specific\MRITC_RAW_STILLS')
files = glob.glob(sourcedir+os.path.sep+'*.JPG',recursive=False)
print(len(files))


656


# 4 generate the new names and path for the images

In [139]:
exif = pd.DataFrame([ process_exifdata(file) for file in files])
exif.index =log.make_unique(exif)
exif.sort_index(inplace=True)
exif['Platform']=log.deployment_metadata['Platform']
exif['Voyage']=log.deployment_metadata['Voyage']
exif['Operation_identifier']=log.deployment_metadata['Operation_identifier']

exif =exif.merge(log.camera_metadata,left_on=['bodySerialNumber'],right_on=['Body_serial_number'],how='left').set_index(exif.index)
exif.drop(['Body_serial_number'],inplace=True,axis=1)
#ex.index =exif.index
# set process flat for avalable images
exif['image_count']=1
exif['process'] =(exif.index > starttime) & (exif.index <= endtime) & (~exif.bodySerialNumber.isna())
exif['delta'] = (exif['timestamp']-exif['timestamp'].shift()).fillna(0)
exif.loc[exif.process,'image_count']=(~(exif.loc[exif.process,'delta']<pd.Timedelta(2000,'ms'))).cumsum()
if exif.loc[exif.process,'image_count'].min()==0:
     exif.loc[exif.process,'image_count'] = exif.loc[exif.process,'image_count'] +1
exif.loc[~exif.process,'image_count']=(~(exif.loc[~exif.process,'delta']<pd.Timedelta(2000,'ms'))).cumsum()
#exif['image_count']=exif['image_count']+
exif['pair']=1
exif['pair']=exif.groupby('image_count')['pair'].transform(lambda x: (x.sum()))
destinationdir = r'Z:\voyage_specific\\'
destinationroot  = destinationdir +log.deployment_metadata['Platform']+os.path.sep \
                    +log.deployment_metadata['Voyage']+'_' \
                    +str(log.deployment_metadata['Operation_identifier']).zfill(3)+os.path.sep
stillsdestination = destinationroot+'stills'+os.path.sep

# find all the unalocated camera names and put the serial number in.
exif.loc[exif.Camera_name.isna(),'Camera_name'] = exif[exif.Camera_name.isna()]['bodySerialNumber'].astype(str)

exif['destination'] = stillsdestination #stillsdestination+exif['Camera_name'].apply(str)+os.path.sep

# Set the new file to the old file name (deafult)

def make_file_name(item):
    item.new_file = item.Platform + '_'+ item.Camera_name+'_' + item.Voyage + '_'+ \
                        str(item.Operation_identifier).zfill(3) + '_'+ item.name.strftime('%Y%m%dT%H%M%SZ')+\
                        '_'+str(item.image_count).zfill(4)+os.path.splitext(os.path.split(item.origonal_file)[1])[1]
    return item

# make a sequential number for the image 
if len(exif)>0:
    exif['new_file'] = ''
    exif.sort_index(inplace=True)
    exif =exif.apply(make_file_name,axis=1)
    exif['destination'] = stillsdestination
    exif.loc[~exif.process,'destination'] = exif.loc[~exif.process,'destination']+r"\novideo\\"
    exif['new_file'] = ''
    exif.sort_index(inplace=True)
    exif =exif.apply(make_file_name,axis=1)
    exif.loc[exif['pair']==1,'destination']=exif.loc[exif['pair']==1,'destination']+r"\orphan\\"
    dirstomake =exif['destination'].unique()
    for dirs in dirstomake:
        os.makedirs(dirs, exist_ok=True)

        
        
# let's highlight anthing that's not going to be moved

def highlight_vals(val, min=-0.1, max=10, color='green'):
    if val:
        return 'background-color: %s' % 'green'
    else:
        return 'background-color: %s' % 'red'

exif.style.applymap(highlight_vals, subset=['process'])
#print(exif.head(-1))

,bodySerialNumber,cameraOwnerName,lens,origonal_file,origonal_file_hash,process,timestamp,dup,Platform,Voyage,Operation_identifier,Camera_name,Description,image_count,delta,pair,destination,new_file
timestamp,,,,,,,,,,,,,,,,,,
2018-12-14 11:47:45.360000,75012000115,Deep Tow Camera STBD,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4948.JPG,None,False,2018-12-14 11:47:45.360000,False,MRITC,IN2018_V06,165,SCS,Starboard stereo camera,0,0 days 00:00:00,1,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\\novideo\\\orphan\\,MRITC_SCS_IN2018_V06_165_20181214T114745Z_0000.JPG
2018-12-14 11:47:50.340000,95013000014,Deep Tow Camera PORT,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3287.JPG,None,False,2018-12-14 11:47:50.340000,False,MRITC,IN2018_V06,165,SCP,Port stereo camera,1,0 days 00:00:04.980000,3,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\\novideo\\,MRITC_SCP_IN2018_V06_165_20181214T114750Z_0001.JPG
2018-12-14 11:47:55.140000,75012000115,Deep Tow Camera STBD,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4949.JPG,None,False,2018-12-14 11:47:55.140000,False,MRITC,IN2018_V06,165,SCS,Starboard stereo camera,2,0 days 00:00:04.800000,4,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\\novideo\\,MRITC_SCS_IN2018_V06_165_20181214T114755Z_0002.JPG
2018-12-14 11:47:55.330000,95013000014,Deep Tow Camera PORT,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3288.JPG,None,False,2018-12-14 11:47:55.330000,False,MRITC,IN2018_V06,165,SCP,Port stereo camera,2,0 days 00:00:00.190000,4,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\\novideo\\,MRITC_SCP_IN2018_V06_165_20181214T114755Z_0002.JPG
2018-12-14 11:48:00.150000,75012000115,Deep Tow Camera STBD,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4950.JPG,None,False,2018-12-14 11:48:00.150000,False,MRITC,IN2018_V06,165,SCS,Starboard stereo camera,3,0 days 00:00:04.820000,4,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\\novideo\\,MRITC_SCS_IN2018_V06_165_20181214T114800Z_0003.JPG
2018-12-14 11:48:00.340000,95013000014,Deep Tow Camera PORT,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3289.JPG,None,False,2018-12-14 11:48:00.340000,False,MRITC,IN2018_V06,165,SCP,Port stereo camera,3,0 days 00:00:00.190000,4,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\\novideo\\,MRITC_SCP_IN2018_V06_165_20181214T114800Z_0003.JPG
2018-12-14 12:04:28.180000,75012000115,Deep Tow Camera STBD,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4951.JPG,None,True,2018-12-14 12:04:28.180000,False,MRITC,IN2018_V06,165,SCS,Starboard stereo camera,1,0 days 00:16:27.840000,3,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\,MRITC_SCS_IN2018_V06_165_20181214T120428Z_0001.JPG
2018-12-14 12:04:28.370000,95013000014,Deep Tow Camera PORT,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3290.JPG,None,True,2018-12-14 12:04:28.370000,False,MRITC,IN2018_V06,165,SCP,Port stereo camera,1,0 days 00:00:00.190000,3,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\,MRITC_SCP_IN2018_V06_165_20181214T120428Z_0001.JPG
2018-12-14 12:04:33.170000,75012000115,Deep Tow Camera STBD,18mm,Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4952.JPG,None,True,2018-12-14 12:04:33.170000,False,MRITC,IN2018_V06,165,SCS,Starboard stereo camera,2,0 days 00:00:04.800000,4,Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\,MRITC_SCS_IN2018_V06_165_20181214T120433Z_0002.JPG


In [140]:
exif.loc[exif['process']].groupby('Camera_name').count()

,bodySerialNumber,cameraOwnerName,lens,origonal_file,origonal_file_hash,process,timestamp,dup,Platform,Voyage,Operation_identifier,Description,image_count,delta,pair,destination,new_file
Camera_name,,,,,,,,,,,,,,,,,
SCP,325,325,325,325,0,325,325,325,325,325,325,325,325,325,325,325,325
SCS,325,325,325,325,0,325,325,325,325,325,325,325,325,325,325,325,325


# 5 move the images across

In [141]:
#dirtomake =exif['destination'].unique()
#for dirname in dirtomake:
#    os.makedirs(dirname, exist_ok=True)

def copy_item(item,procall=False,inplace=False):
    if (item.process | procall):
        if inplace:
            dest =os.path.split(item.origonal_file)[0]+os.path.sep+item.new_file
        else:
            dest =item.destination+item.new_file
        print(item.origonal_file,dest+item.new_file)
        if os.path.exists(item.origonal_file):
            print('moving')
            try:
                shutil.move(item.origonal_file,dest)
            except:
                print(item.origonal_file,item.destination+item.new_file)
       # shutil.copy2(item.origonal_file,item.destination+item.new_file)

def do_afer(item):
    dest =item.destination+item.new_file
    source =os.path.split(item.origonal_file)[0]+os.path.sep+item.new_file
    print(source,dest)
    if os.path.exists(source):
        try:
            shutil.move(source,dest)
        except:
            print('Error ')
    
movecount =len(exif.apply(copy_item,axis=1,procall=False));


#movecount =len(exif[exif.process].apply(do_afer,axis=1));

Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4951.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120428Z_0001.JPGMRITC_SCS_IN2018_V06_165_20181214T120428Z_0001.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3290.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120428Z_0001.JPGMRITC_SCP_IN2018_V06_165_20181214T120428Z_0001.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4952.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120433Z_0002.JPGMRITC_SCS_IN2018_V06_165_20181214T120433Z_0002.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3291.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120433Z_0002.JPGMRITC_SCP_IN2018_V06_165_20181214T120433Z_0002.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4953.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120438Z_0003.JPGMRITC_SCS_IN2

Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3314.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120638Z_0025.JPGMRITC_SCP_IN2018_V06_165_20181214T120638Z_0025.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4976.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120643Z_0026.JPGMRITC_SCS_IN2018_V06_165_20181214T120643Z_0026.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3315.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120643Z_0026.JPGMRITC_SCP_IN2018_V06_165_20181214T120643Z_0026.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4977.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120648Z_0027.JPGMRITC_SCS_IN2018_V06_165_20181214T120648Z_0027.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3316.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120648Z_0027.JPGMRITC_SCP_IN2

Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3335.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120823Z_0046.JPGMRITC_SCP_IN2018_V06_165_20181214T120823Z_0046.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4997.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120828Z_0047.JPGMRITC_SCS_IN2018_V06_165_20181214T120828Z_0047.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3336.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120828Z_0047.JPGMRITC_SCP_IN2018_V06_165_20181214T120828Z_0047.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD4998.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T120833Z_0048.JPGMRITC_SCS_IN2018_V06_165_20181214T120833Z_0048.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3337.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T120833Z_0048.JPGMRITC_SCP_IN2

Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5027.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T121103Z_0077.JPGMRITC_SCS_IN2018_V06_165_20181214T121103Z_0077.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3366.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T121103Z_0077.JPGMRITC_SCP_IN2018_V06_165_20181214T121103Z_0077.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5028.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T121108Z_0078.JPGMRITC_SCS_IN2018_V06_165_20181214T121108Z_0078.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3367.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T121108Z_0078.JPGMRITC_SCP_IN2018_V06_165_20181214T121108Z_0078.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5029.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T121113Z_0079.JPGMRITC_SCS_IN2

Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3396.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T121333Z_0107.JPGMRITC_SCP_IN2018_V06_165_20181214T121333Z_0107.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5058.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T121338Z_0108.JPGMRITC_SCS_IN2018_V06_165_20181214T121338Z_0108.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3397.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T121338Z_0108.JPGMRITC_SCP_IN2018_V06_165_20181214T121338Z_0108.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5059.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T121343Z_0109.JPGMRITC_SCS_IN2018_V06_165_20181214T121343Z_0109.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3398.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T121343Z_0109.JPGMRITC_SCP_IN2

...

moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5144.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122103Z_0194.JPGMRITC_SCS_IN2018_V06_165_20181214T122103Z_0194.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3483.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122103Z_0194.JPGMRITC_SCP_IN2018_V06_165_20181214T122103Z_0194.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5145.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122113Z_0195.JPGMRITC_SCS_IN2018_V06_165_20181214T122113Z_0195.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3484.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122113Z_0195.JPGMRITC_SCP_IN2018_V06_165_20181214T122113Z_0195.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5146.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122118Z_0196.JPGMRITC_

Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5181.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122418Z_0231.JPGMRITC_SCS_IN2018_V06_165_20181214T122418Z_0231.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3520.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122418Z_0231.JPGMRITC_SCP_IN2018_V06_165_20181214T122418Z_0231.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5182.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122423Z_0232.JPGMRITC_SCS_IN2018_V06_165_20181214T122423Z_0232.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3521.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122423Z_0232.JPGMRITC_SCP_IN2018_V06_165_20181214T122423Z_0232.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5183.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122428Z_0233.JPGMRITC_SCS_IN2

moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3550.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122653Z_0261.JPGMRITC_SCP_IN2018_V06_165_20181214T122653Z_0261.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5212.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122658Z_0262.JPGMRITC_SCS_IN2018_V06_165_20181214T122658Z_0262.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3551.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122658Z_0262.JPGMRITC_SCP_IN2018_V06_165_20181214T122658Z_0262.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5213.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122703Z_0263.JPGMRITC_SCS_IN2018_V06_165_20181214T122703Z_0263.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3552.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122703Z_0263.JPGMRITC_

Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3570.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122838Z_0282.JPGMRITC_SCP_IN2018_V06_165_20181214T122838Z_0282.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5232.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122843Z_0283.JPGMRITC_SCS_IN2018_V06_165_20181214T122843Z_0283.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3571.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122843Z_0283.JPGMRITC_SCP_IN2018_V06_165_20181214T122843Z_0283.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5233.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T122848Z_0284.JPGMRITC_SCS_IN2018_V06_165_20181214T122848Z_0284.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3572.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T122848Z_0284.JPGMRITC_SCP_IN2

moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5262.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T123118Z_0313.JPGMRITC_SCS_IN2018_V06_165_20181214T123118Z_0313.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3601.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T123118Z_0313.JPGMRITC_SCP_IN2018_V06_165_20181214T123118Z_0313.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5263.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T123123Z_0314.JPGMRITC_SCS_IN2018_V06_165_20181214T123123Z_0314.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\A14I3602.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCP_IN2018_V06_165_20181214T123123Z_0314.JPGMRITC_SCP_IN2018_V06_165_20181214T123123Z_0314.JPG
moving
Z:/voyage_specific/MRITC_RAW_STILLS/165\STBD5264.JPG Z:\voyage_specific\\MRITC\IN2018_V06_165\stills\MRITC_SCS_IN2018_V06_165_20181214T123128Z_0315.JPGMRITC_

In [105]:
movecount

1506